In [1]:
import fastf1
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import plotly.express as px
import numpy as np

YEARS = [2020, 2021, 2022, 2023, 2024, 2025]
SESSION = 'R'
ALL_TRACKS = [
    'Bahrain', 'Saudi Arabia', 'Australia', 'Japan', 'China', 'Miami',
    'Emilia Romagna', 'Monaco', 'Canada', 'Spain', 'Austria', 'Great Britain',
    'Hungary', 'Belgium', 'Netherlands', 'Italy', 'Azerbaijan', 'Singapore',
    'United States', 'Mexico', 'Brazil', 'Las Vegas', 'Qatar', 'Abu Dhabi',
    'France', 'Portugal', 'Turkey', 'Russia', 'Eifel' 
]
COMPOUNDS = ['SOFT', 'MEDIUM', 'HARD']

In [2]:
all_laps = []
print(f"Starting data collection for {len(YEARS)} seasons across {len(ALL_TRACKS)} tracks...")
for year in YEARS:
    # Get the schedule for this year to know which tracks were actually raced
    try:
        schedule = fastf1.get_event_schedule(year)
        # Filter for completed official races
        races = schedule[schedule['EventFormat'] != 'testing']['EventName'].unique()
    except:
        print(f"Could not fetch schedule for {year}")
        continue

    for track_name in races:
        # Map official name to our list (simplified matching)
        # In a real app, you might need a more robust name mapper
        matched_track = None
        for t in ALL_TRACKS:
            if t in track_name:
                matched_track = t
                break
        
        if not matched_track: continue # Skip if track not in our main list

        try:
            session = fastf1.get_session(year, track_name, SESSION)
            session.load(telemetry=False, weather=False, messages=False)
            
            # Get valid racing laps
            laps = session.laps.pick_quicklaps().dropna(subset=['LapTime', 'TyreLife', 'Compound'])
            
            if len(laps) < 100: continue # Skip if bad data (e.g. rain race with no dry laps)

            # NORMALIZE: Calculate Pace Delta vs. The Winner's Pace
            # This makes a 2020 lap comparable to a 2024 lap
            best_lap = laps['LapTime'].dt.total_seconds().min()
            laps['PaceDelta'] = laps['LapTime'].dt.total_seconds() - best_lap
            laps['Track'] = matched_track
            
            # Filter for dry tires
            laps = laps[laps['Compound'].isin(COMPOUNDS)]
            
            all_laps.append(laps[['Compound', 'TyreLife', 'PaceDelta', 'Track']])
            print(f"  [OK] Loaded {year} {matched_track} ({len(laps)} laps)")
            
        except Exception as e:
            # Silence errors for cancelled races or data gaps
            pass

if not all_laps:
    raise ValueError("No data found! Please check your connection.")

req         WARNING 	DEFAULT CACHE ENABLED! (786.65 MB) /Users/iwosmura/Library/Caches/fastf1


Starting data collection for 6 seasons across 29 tracks...


core           INFO 	Loading data for Austrian Grand Prix - Race [v3.7.0]
req            INFO 	No cached data found for session_info. Loading data...
_api           INFO 	Fetching session info data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for driver_info. Loading data...
_api           INFO 	Fetching driver list...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for session_status_data. Loading data...
_api           INFO 	Fetching session status data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for lap_count. Loading data...
_api           INFO 	Fetching lap count data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for track_status_data. Loading data...
_api           INFO 	Fetching track status data...
req            INFO 	Data has been written to cache!
req            INFO 	No c

  [OK] Loaded 2020 Austria (775 laps)


req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for driver_info. Loading data...
_api           INFO 	Fetching driver list...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for session_status_data. Loading data...
_api           INFO 	Fetching session status data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for lap_count. Loading data...
_api           INFO 	Fetching lap count data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for track_status_data. Loading data...
_api           INFO 	Fetching track status data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for _extended_timing_data. Loading data...
_api           INFO 	Fetching timing data...
_api           INFO 	Parsing timing data...
req            INFO 	Data has been written to cache!

...

  [OK] Loaded 2025 Las Vegas (767 laps)


Request for URL https://api.jolpi.ca/ergast/f1/2025/23/results.json failed; using cached response
Traceback (most recent call last):
  File "/Users/iwosmura/projects/f1fun/venv/lib/python3.14/site-packages/requests_cache/session.py", line 291, in _resend
    response.raise_for_status()
    ~~~~~~~~~~~~~~~~~~~~~~~~~^^
  File "/Users/iwosmura/projects/f1fun/venv/lib/python3.14/site-packages/requests/models.py", line 1026, in raise_for_status
    raise HTTPError(http_error_msg, response=self)
requests.exceptions.HTTPError: 429 Client Error: Too Many Requests for url: https://api.jolpi.ca/ergast/f1/2025/23/results.json
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
Request for URL https://api.j

  [OK] Loaded 2025 Qatar (888 laps)


Request for URL https://api.jolpi.ca/ergast/f1/2025/24/results.json failed; using cached response
Traceback (most recent call last):
  File "/Users/iwosmura/projects/f1fun/venv/lib/python3.14/site-packages/requests_cache/session.py", line 291, in _resend
    response.raise_for_status()
    ~~~~~~~~~~~~~~~~~~~~~~~~~^^
  File "/Users/iwosmura/projects/f1fun/venv/lib/python3.14/site-packages/requests/models.py", line 1026, in raise_for_status
    raise HTTPError(http_error_msg, response=self)
requests.exceptions.HTTPError: 429 Client Error: Too Many Requests for url: https://api.jolpi.ca/ergast/f1/2025/24/results.json
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
core           INFO 	Finished

  [OK] Loaded 2025 Abu Dhabi (1103 laps)


In [3]:
df = pd.concat(all_laps)
print(f"\nTraining on {len(df)} total data points...")
X_list, y_list = [], []

for _, row in df.iterrows():
    features = [row['TyreLife']]
    comp_vec = [0] * len(COMPOUNDS)
    if row['Compound'] in COMPOUNDS:
        comp_vec[COMPOUNDS.index(row['Compound'])] = 1
    features.extend(comp_vec)

    track_vec = [0] * len(ALL_TRACKS)
    if row['Track'] in ALL_TRACKS:
        track_vec[ALL_TRACKS.index(row['Track'])] = 1
    features.extend(track_vec)

    X_list.append(features)
    y_list.append(row['PaceDelta'])
X, y = torch.tensor(X_list, dtype=torch.float32), torch.tensor(y_list, dtype=torch.float32).reshape(-1, 1)

class TireNet(nn.Module):
    def __init__(self, input_size):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_size, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 1)
        )
    def forward(self, x):
        return self.net(x)
input_size = 1 + len(COMPOUNDS) + len(ALL_TRACKS)
model = TireNet(input_size)
optimizer = optim.Adam(model.parameters(), lr = 0.001)
criterion = nn.MSELoss()


Training on 60454 total data points...


In [4]:
for epoch in range(5000):
    preds = model(X)
    loss = criterion(preds, y)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if epoch % 500 == 0:
        print(f"Epoch {epoch}: Loss {loss.item():.4f}")

Epoch 0: Loss 11.5984
Epoch 500: Loss 1.4074
Epoch 1000: Loss 1.3988
Epoch 1500: Loss 1.3938
Epoch 2000: Loss 1.3909
Epoch 2500: Loss 1.3889
Epoch 3000: Loss 1.3874
Epoch 3500: Loss 1.3875
Epoch 4000: Loss 1.3861
Epoch 4500: Loss 1.3856


In [15]:
TEST_TRACK = 'Monaco'
print(f'Generating predictions for {TEST_TRACK}...')

viz_data = []
ages = np.arange(0, 50, 1)

with torch.no_grad():
    track_idx = ALL_TRACKS.index(TEST_TRACK)

    for comp_name in COMPOUNDS:
        comp_idx = COMPOUNDS.index(comp_name)

        batch_inputs = []

        for age in ages:
            vec = [age] + [0]*len(COMPOUNDS) + [0]*len(ALL_TRACKS)
            vec[1+ comp_idx] = 1
            vec[1 + len(COMPOUNDS) + track_idx] = 1
            batch_inputs.append(vec)
        preds = model(torch.tensor(batch_inputs, dtype=torch.float32)).numpy().flatten()

        for age, val in zip(ages, preds):
            viz_data.append({
                'Tire Age': age,
                'Compound': comp_name,
                'Predicted Time Loss (s)': val
            })
df_viz = pd.DataFrame(viz_data)

fig = px.line(
    df_viz, x='Tire Age', y='Predicted Time Loss (s)', color='Compound',
    title=f'AI Prediction: Tire Degradation at {TEST_TRACK} (Model trained on 2020-2025 data)',
    template="plotly_dark",
    color_discrete_map={'SOFT': 'red', 'MEDIUM': 'yellow', 'HARD': 'white'}
)
fig.update_layout(yaxis_title="Seconds Slower vs Quali Pace")
fig.show()

Generating predictions for Monaco...
